# Read HDF5 File

*Author: Creare* <br>
*Date: April 01 2020* <br>

**Keywords**: podpac, DataSource, HDF5, netCDF

## Overview

Reference tutorial for loading data from an HDF5 formatted (i.e. netCDF) file using the `podpac.data.H5PY` Node.

### Prerequisites

- Python 2.7 or above
- [`podpac[datatype]`](https://podpac.org/install.html#install)
- *Review the [README.md](../README.md) and [jupyter-tutorial.ipynb](jupyter-tutorial.ipynb) for additional info on using jupyter notebooks*

### See Also

- [0-concepts/introduction.ipynb](introduction.ipynb): PODPAC introduction
- [0-concepts/coordinates.ipynb](../0-concepts/coordinates.ipynb): PODPAC Coordinates reference
- [csv-file.ipynb](csv-file.ipynb): PODPAC CSV DataSource
- [`podpac.data.H5PY` API Reference](https://podpac.org/api/podpac.data.H5PY.html#podpac.data.H5PY)

## Open Simple HDF5 file

The `podpac.data.H5PY` node reads HDF5 formatted files.
PODPAC wraps the [h5py](https://www.h5py.org/) Python package to read HDF5 files.

If the HDF5 file contains tabular data (like a CSV file), then DataSource should work out of the box similar to a CSV file.

* Specify the column used for the data (`data_key`)
* Specify the column(s) used for the coordinates:
    * `lat_key`
    * `lon_key`
    * `time_key`
    * `alt_key`

In [ ]:
import podpac

# Get the file path to the SMAP data in the podpac_examples repostiory
filename = '../../data/'  # TODO: find netcdf file to use here

# Set up the PODPAC node to read this file. 
node = podpac.data.H5PY(source=filename, data_key="", lat_key="", lon_key="")
node

In [ ]:
# retrieve all of the data at the native resolution into memory and plot
output = node.eval(node.native_coordinates)
output.plot()
pass

## Open Complex HDF5 file

If the data in your HDF5 file is not compatible with the `podpac.data.H5PY` node out-of-the-box (i.e. it is not tabular-like), you will have to create a custom Node to interpret the data in your file.
  
This takes a little more work and is *and is signifigantly more complicated*.
We encourage users to contribute wrappers for datasets. 
We welcome pull requests to the `podpac.datalib` module of the PODPAC repository.
**One of PODPAC's goals is to share the work of wrapping important datasets.**

With that caveat, lets put together a Node to interpret a SMAP formatted HDF5 file: `'../../data/raster/SMAP_L4_SM_aup_20181027T090000_Vv4030_001.h5'`

So, we have to:

1. Find the *keys* for the data we want
2. Tell PODPAC what are the coordinates of the data

The example below is relatively streamlined, but each new dataset may require some iteration. For example, you'll have to consider the: 

* Order of the coordinates 
    * (lat, lon) vs (lon, lat)
* The dimension of the array
    * Is this a 2D array with lat-lon dimensions? 
    * Is this a 3D array with lat-lon-time dimensions?

In [1]:
import podpac

# Get the file path to the SMAP data in the podpac_examples repostiory
filename = '../../data/raster/SMAP_L4_SM_aup_20181027T090000_Vv4030_001.h5'

# Set up the PODPAC node to read this file so we can investigate the keys.
# Notice, we specify that the number -9999 should be interpreted as a 'nan' in Python.
node = podpac.data.H5PY(source=filename, nan_vals=[-9999])
node

<H5PY(source='../../data/raster/SMAP_L4_SM_aup_20181027T090000_Vv4030_001.h5', interpolation='nearest')>

## View keys

In [2]:
node.keys

['/Analysis_Data/sm_profile_analysis',
 '/Analysis_Data/sm_profile_analysis_ensstd',
 '/Analysis_Data/sm_rootzone_analysis',
 '/Analysis_Data/sm_rootzone_analysis_ensstd',
 '/Analysis_Data/sm_surface_analysis',
 '/Analysis_Data/sm_surface_analysis_ensstd',
 '/Analysis_Data/soil_temp_layer1_analysis',
 '/Analysis_Data/soil_temp_layer1_analysis_ensstd',
 '/Analysis_Data/surface_temp_analysis',
 '/Analysis_Data/surface_temp_analysis_ensstd',
 '/EASE2_global_projection',
 '/Forecast_Data/sm_profile_forecast',
 '/Forecast_Data/sm_rootzone_forecast',
 '/Forecast_Data/sm_surface_forecast',
 '/Forecast_Data/soil_temp_layer1_forecast',
 '/Forecast_Data/surface_temp_forecast',
 '/Forecast_Data/tb_h_forecast',
 '/Forecast_Data/tb_h_forecast_ensstd',
 '/Forecast_Data/tb_v_forecast',
 '/Forecast_Data/tb_v_forecast_ensstd',
 '/Observations_Data/tb_h_obs',
 '/Observations_Data/tb_h_obs_assim',
 '/Observations_Data/tb_h_obs_errstd',
 '/Observations_Data/tb_h_obs_time_sec',
 '/Observations_Data/tb_h_or

The keys suggests we want `cell_lat` and `cell_lon` keys for `lat` and `lon` coordinates

We are interested in surface soil moisture, so let's use the `/Analysis_Data/sm_surface_analysis` key for our data.

We'll make a new DataSource class that tells PODPAC how to interpret the coordinates and data from the file.

In [3]:
# Define a new class, inheriting from the PODPAC H5PY class
class SMAPH5(podpac.data.H5PY):
    # key for data values
    data_key = '/Analysis_Data/sm_surface_analysis'
    
    # Overwrite the 'get_native_coordinates' function of podpac.data.H5PY
    # to tell PODPAC how to assemble coordinates
    def get_native_coordinates(self):
        lat = self.dataset['cell_lat'][:, 0]
        lon = self.dataset['cell_lon'][0, :]
        
        # the order is important: lat = rows of array, lon = cols of array
        return podpac.Coordinates([lat, lon], dims=['lat', 'lon'])

In [4]:
# Make a PODPAC node to read this file
node = SMAPH5(source=filename, nan_vals=[-9999])
node.native_coordinates

Coordinates (EPSG:4326)
	lat: ArrayCoordinates1d(lat): Bounds[-84.65644073486328, 84.65644073486328], N[1624], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[-179.9533233642578, 179.9533233642578], N[3856], ctype['midpoint']

In [5]:
# retrieve all of the data at the native resolution into memory and plot
output = node.eval(node.native_coordinates)
output.plot()
pass

TypeError: 'method' object is not iterable

> TODO: this should move to 2- combining data

##  Example of automatic interpolation

Once the dataset is read through PODPAC, it can be re-interpolated automatically to a different coordinate system.

The default interpolation scheme is nearest-neighbor ("nearest"). 

Let's read **a subset of the SMAP data**, using nearest-neighbor interpolation **at the resolution of the SRTM file**. 
See [raster-file.ipynb](raster-file.ipynb) for example reading the SRTM Geotif file.

In [ ]:
import podpac

srtm_filename = '../../data/raster/n39_w107_1arc_v2.tif'
srtm = podpac.data.Rasterio(source=srtm_filename)

node = SMAPH5(source=filename, nan_vals=[-9999], interpolation="bilinear")

In [ ]:
output = node.eval(srtm.native_coordinates)
output.plot()
pass